# Übung 4 - UV-Decomposition

Lese im Skrip den Abschnitt 9.4: Dimensionality Reduction durch, und implementiere den UV-Decompositions-Algorithmus

In [1]:
import numpy as np

In [2]:
# initialisiere M und U,V
M = np.array([
    [5,2,4,4,3],
    [3,1,2,4,1],
    [2,np.nan,3,1,4],
    [2,5,4,3,5],
    [4,4,5,4,np.nan]
])
U = np.ones((5,2))
V = np.ones((2,5))

In [3]:
# UV
P = U @ V
P

array([[2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.],
       [2., 2., 2., 2., 2.]])

In [4]:
# root mean squared error
def rmse(A, B):
    return np.sqrt(np.nanmean(np.power((A - B),2)))

# sum squared error
def sse(A, B):
    return np.nansum(np.power((A - B),2))

In [5]:
# berechne RMSE und SSE
print(rmse(M,P))
print(sse(M,P))

1.805787796286538
75.0


In [6]:
# minimiere U[r,s]
def minimize_U(M, U, V, r, s):       
    U = U.copy()
    a = 0
    b = 0
    for j in range(M.shape[1]):        
        if np.isnan(M[r,j]):
            continue
        a += V[s,j] * (M[r,j] - sum(U[r,k] * V[k,j] for k in range(U.shape[1]) if k != s))
        b += np.power(V[s,j],2) 
    U[r,s] = a/b
    return U


In [7]:
# minimiere U[0,0]
U = minimize_U(M, U, V, 0, 0)

In [8]:
U

array([[2.6, 1. ],
       [1. , 1. ],
       [1. , 1. ],
       [1. , 1. ],
       [1. , 1. ]])

In [9]:
P = U @ V
P

array([[3.6, 3.6, 3.6, 3.6, 3.6],
       [2. , 2. , 2. , 2. , 2. ],
       [2. , 2. , 2. , 2. , 2. ],
       [2. , 2. , 2. , 2. , 2. ],
       [2. , 2. , 2. , 2. , 2. ]])

In [10]:
print(rmse(M,P))
print(sse(M,P))

1.6444901416812923
62.2


In [11]:
# minimiere V[r,s]
def minimize_V(M, U, V, r, s):       
    V = V.copy()
    a = 0
    b = 0
    for i in range(M.shape[0]):        
        if np.isnan(M[i,s]):
            continue
        a += U[i,r] * (M[i,s] - sum(U[i,k] * V[k,s] for k in range(V.shape[0]) if k != r))
        b += np.power(U[i,r],2) 
    V[r,s] = a/b
    return V

In [12]:
# minimiere V[0,0]
V = minimize_V(M, U, V, 0, 0)
V

array([[1.61710037, 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ]])

In [13]:
P = U @ V
P

array([[5.20446097, 3.6       , 3.6       , 3.6       , 3.6       ],
       [2.61710037, 2.        , 2.        , 2.        , 2.        ],
       [2.61710037, 2.        , 2.        , 2.        , 2.        ],
       [2.61710037, 2.        , 2.        , 2.        , 2.        ],
       [2.61710037, 2.        , 2.        , 2.        , 2.        ]])

In [14]:
print(rmse(M,P))
print(sse(M,P))

1.5894004000907482
58.10245353159851


In [15]:
# minimiere U[2,0] mit fehlenden Werten
U = minimize_U(M, U, V, 2, 0)

In [16]:
P = U @ V
P

array([[5.20446097, 3.6       , 3.6       , 3.6       , 3.6       ],
       [2.61710037, 2.        , 2.        , 2.        , 2.        ],
       [2.90569716, 2.1784656 , 2.1784656 , 2.1784656 , 2.1784656 ],
       [2.61710037, 2.        , 2.        , 2.        , 2.        ],
       [2.61710037, 2.        , 2.        , 2.        , 2.        ]])

In [17]:
print(rmse(M,P))
print(sse(M,P))

1.5869524460602322
57.92361551930075


In [18]:
# UV-Decomposition Algorithmus
def UV_decomposition(M, k, delta_treshold=0.0001, max_iter=50):
    init_value = np.sqrt(np.nanmean(M)/k)
    U = np.ones((M.shape[0],k)) * init_value
    V = np.ones((k,M.shape[1])) * init_value
    
    delta = np.inf
    last_error = np.inf
    i = 0
    while delta > delta_treshold and i < max_iter:
        for r in range(U.shape[0]):
            for s in range(U.shape[1]):
                U = minimize_U(M, U, V, r, s)
            print(f'{i:>2}: {rmse(M,U @ V):.7}', end="\r")

        for r in range(V.shape[0]):
            for s in range(V.shape[1]):
                V = minimize_V(M, U, V, r, s)
            print(f'{i:>2}: {rmse(M,U @ V):.7}', end="\r")

        P = U @ V
        error = rmse(M,P)
        delta = last_error - error
        last_error = error
        print(f'{i:>2}: {error:.7}')
        i += 1
    return U, V

In [19]:
U, V = UV_decomposition(M, 2)

 0: 0.9859717
 1: 0.9419209
 2: 0.7800956
 3: 0.5041983
 4: 0.3796521
 5: 0.3481008
 6: 0.3380747
 7: 0.3344906
 8: 0.3331675
 9: 0.3326725
10: 0.3324799
11: 0.3324062


In [20]:
P = U @ V
P

array([[4.53618242, 2.21942388, 3.87397298, 4.5500183 , 2.85874719],
       [3.38885061, 0.65381128, 2.31222815, 3.4179518 , 1.15032115],
       [1.50940223, 3.79888173, 3.06235365, 1.45682213, 3.95404955],
       [2.47240693, 4.80014421, 4.19193104, 2.41285919, 5.08106563],
       [4.08678248, 4.16201533, 4.7431972 , 4.05884128, 4.69732504]])

In [21]:
print(rmse(M,P))
print(sse(M,P))

0.3324062215870878
2.5413596114454955


In [22]:
M

array([[ 5.,  2.,  4.,  4.,  3.],
       [ 3.,  1.,  2.,  4.,  1.],
       [ 2., nan,  3.,  1.,  4.],
       [ 2.,  5.,  4.,  3.,  5.],
       [ 4.,  4.,  5.,  4., nan]])